In [2]:
from ResattUnet import *
from data import *
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [ ]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'dataset/train','image','label',data_gen_args,save_to_dir = None)
validGene= trainGenerator(2,'dataset/validation','image','label',data_gen_args,save_to_dir = None)
model = Attention_ResUNet()
reduced_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, mode='auto')
earlystopper = EarlyStopping(patience=5, verbose=1)
# get u_net model
model = Attention_ResUNet()
model_checkpoint = ModelCheckpoint('ResattUnet.hdf5', monitor='val_loss',verbose=1, save_best_only=True)
hist = model.fit_generator(myGene, steps_per_epoch=20, epochs=34, callbacks=[reduced_lr, model_checkpoint, earlystopper], 
                     validation_data=validGene, validation_steps=3)


### Train with npy file

In [10]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [5]:
testGene = testGenerator('dataset/test/image')
model = Attention_ResUNet()
model.load_weights("hh.hdf5")
results = model.predict_generator(testGene,3,verbose=1)
results[results>0.5]=1

results[results<=0.5]=0
saveResult("dataset/test/results",results)


3/3 [==============================] - 9s 3s/step


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [6]:
print(results[0,:,:,:])

[[[1.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[1.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[1.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [1.]]]


In [13]:
img=io.imread('./dataset/test/label/0.png')
print(img)

[[255 255 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [14]:
import matplotlib.pyplot as plt
img = io.imread('./dataset/test/label/0.png')
img = img.astype(np.float32)
img1 = trans.resize(img,[256,256])

In [15]:
result = io.imread('./dataset/test/results/0_predict.png')
img_test=img.astype(np.float32)

In [ ]:
mean1=dice_coef(img1/255, results[0,:,:,:])
print("Mean Dice Coeff : ",mean1)

In [ ]:
# img = io.imread('./dataset/test/label/1.png')
img = img.astype(np.float32)
img2 = trans.resize(img,[256,256])
img = io.imread('./dataset/test/results/1_predict.png')
img=img.astype(np.float32)
mean2=dice_coef(img2/255, results[1,:,:,:])
print("Mean Dice Coeff : ",mean2)

In [ ]:
img = io.imread('./dataset/test/label/2.png')
img = img.astype(np.float32)
img3 = trans.resize(img,[256,256])
img = io.imread('./dataset/test/results/2_predict.png')
img = img.astype(np.float32)
mean3 = dice_coef(img3/255, results[2,:,:,:])
print("Mean Dice Coeff : ",mean3)

In [ ]:
mean_average=(mean1+mean2+mean3)/3
print(mean_average)

In [ ]:
arr = [mean1, mean2, mean3]

np.std(arr)*sqrt(3)